In [1]:
import pandas as pd 
import psycopg2

In [2]:
conn = psycopg2.connect(host='localhost',
                        dbname='sql_practice',
                        user='postgres',
                        password='Aujaswi@31',
                        port=5432)
conn.autocommit = True
cur = conn.cursor()

# Question

### Bank of Ireland has requested that you detect invalid transactions in December 2022. An invalid transaction is one that occurs outside of the bank's normal business hours. The following are the hours of operation for all branches:
- Monday - Friday 09:00 - 16:00
- Saturday & Sunday Closed
- Irish Public Holidays 25th and 26th December
- Determine the transaction ids of all invalid transactions.

## Creating the table

In [3]:
cur.execute("""CREATE TABLE boi_transactions (transaction_id INT PRIMARY KEY,time_stamp timestamp NOT NULL);""")

## Populating the table

In [4]:
cur.execute("""INSERT INTO boi_transactions (transaction_id, time_stamp) VALUES(1051, '2022-12-03 10:15'),(1052, '2022-12-03 17:00'),(1053, '2022-12-04 10:00'),(1054, '2022-12-04 14:00'),(1055, '2022-12-05 08:59'),(1056, '2022-12-05 16:01'),(1057, '2022-12-06 09:00'),(1058, '2022-12-06 15:59'),(1059, '2022-12-07 12:00'),(1060, '2022-12-08 09:00'),(1061, '2022-12-09 10:00'),(1062, '2022-12-10 11:00'),(1063, '2022-12-10 17:30'),(1064, '2022-12-11 12:00'),(1065, '2022-12-12 08:59'),(1066, '2022-12-12 16:01'),(1067, '2022-12-25 10:00'),(1068, '2022-12-25 15:00'),(1069, '2022-12-26 09:00'),(1070, '2022-12-26 14:00'),(1071, '2022-12-26 16:30'),(1072, '2022-12-27 09:00'),(1073, '2022-12-28 08:30'),(1074, '2022-12-29 16:15'),(1075, '2022-12-30 14:00'),(1076, '2022-12-31 10:00');""")

### Solution

In [5]:
cur.execute("""WITH validity_transactions AS (SELECT transaction_id,time_stamp,
CASE 
WHEN (EXTRACT(DOW FROM time_stamp) IN (0,6)) OR
	 (EXTRACT(HOUR FROM time_stamp) NOT BETWEEN 9 AND 15) OR
	 (EXTRACT(MONTH FROM time_stamp) = 12 AND EXTRACT(DAY FROM time_stamp) in (25,26))
	  THEN 'Invalid' ELSE 'Valid' END AS validity
FROM boi_transactions
WHERE EXTRACT(MONTH FROM time_stamp) = 12 AND EXTRACT(YEAR FROM time_stamp) =2022 )

SELECT transaction_id
FROM validity_transactions
WHERE validity = 'Invalid';""")

In [6]:
df = pd.DataFrame(cur.fetchall(),columns=['transaction_id'])

In [7]:
df

,transaction_id
0,1051
1,1052
2,1053
3,1054
4,1055
5,1056
6,1062
7,1063
8,1064
9,1065


In [8]:
cur.execute("""SELECT table_name
  FROM information_schema.tables
 WHERE table_schema='public'
   AND table_type='BASE TABLE'""")

In [9]:
tables = cur.fetchall()

In [10]:
for i in tables:
    cur.execute(f"""DROP TABLE {i[0]};""")

In [11]:
cur.close()
conn.close()